# Analyse processed Tweets

Problems:

- Commas within text make it difficult to read files > when preprocessing, add a special delimiter to mark the beginning and the start of the Tweet content

In [29]:
# Useful starting lines
%matplotlib inline
import copy
import numpy as np 
#import matplotlib as mpl
#import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import re
import glob
import ast
from datetime import datetime
from collections import Counter
import itertools
import nltk
#nltk.download("stopwords")
from nltk import bigrams 
from nltk.corpus import stopwords
import string
import locale
import reverse_geocoder as rg
locale.setlocale(locale.LC_TIME, "en_US.utf8")
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load and combine Data

In [8]:
#idea read whole line >  then parse
def load_csv_data_old(data_path, sub_sample=False):
    """Loads data and transforms it into a pandas dataframe"""
    
    #extracts tweet ID and timestamp
    tw1 = pd.read_csv(data_path,sep=",",usecols=[0,1],encoding = "ISO-8859-1",header=None)
    
    #extracts tweet content
    #tw2 = pd.read_csv(data_path,sep="[0-9],|,\"\[",usecols=[2],encoding = "ISO-8859-1",header=None,engine="python")
    
    #extract coordinates & location tag
    #tw3 = pd.read_csv(data_path,sep=",\"\[|\]\",",usecols=[1,2],encoding = "ISO-8859-1",header=None,engine="python")
    
    #merge everything
    #tw = pd.concat((tw1,tw2,tw3),axis=1)
    
    #extract complete string per row
    tw = pd.read_csv(data_path,sep="NotADelimiter",usecols=[0,1],encoding = "ISO-8859-1",header=None,engine="python")  
    
    #extract tweet ID and timestamp
    s = tw[0].apply(lambda x: x.split(","))
    tw["id"] = s.apply(lambda x: x[0])
    tw["time"] = s.apply(lambda x: x[1] if len(x) > 1 else None)
    
    #extract tweet content
    s = tw[0].apply(lambda x: re.split("[0-9],|,\"\[",x))
    tw["text"] = s.apply(lambda x: x[2] if len(x) > 2 else None)
    
    #extract coordinates & location tag
    s = tw[0].apply(lambda x: re.split(",\"\[|\]\",",x))
    tw["coord"] = s.apply(lambda x: x[1] if len(x)> 1 else None)
    tw["loc_tag"] = s.apply(lambda x: x[2] if len(x) > 2 else None)
    #tw3 = pd.read_csv(data_path,sep=",\"\[|\]\",",usecols=[1,2],encoding = "ISO-8859-1",header=None,engine="python")
    
    
    # sub-sample
    if sub_sample:
        tw = tw[::50]
    return tw

def load_csv_data(data_path, sub_sample=False):
    """Loads data and transforms it into a pandas dataframe"""
    
    #extracts tweet ID and timestamp
    tw = pd.read_csv(data_path,encoding = "ISO-8859-1",header=None)
      
    # sub-sample
    if sub_sample:
        tw = tw[::50]
    return tw

In [13]:
"""Read all files in Data directory, load & parse them and concatenate resulting DataFrames"""
files = glob.glob('./Data/*.csv')
tw = pd.DataFrame
first = True
for data_path in files:
    tw_temp = load_csv_data(data_path=data_path, sub_sample=True)
    if first:
        tw = tw_temp
        first = False
    else:
        tw = pd.concat((tw,tw_temp),axis=0,ignore_index=False)
tw.index = range(0,np.shape(tw)[0])
tw.columns = ["id","time","text","coord","loc_tag"]

In [14]:
tw

,id,time,text,coord,loc_tag
0,313741620505890817,Mon Mar 18 20:00:06 +0000 2013,got outstanding student of the semester :) sup...,"[-74.2041177, 40.9002544]",coordinates
1,313741627598450689,Mon Mar 18 20:00:07 +0000 2013,Lol jk jk .,"[-74.98284917, 40.09872214]",coordinates
2,313741633277542400,Mon Mar 18 20:00:09 +0000 2013,"Hey @julie_susanne, is it ok if I rent a few m...","[-74.18003082, 40.69879569]",coordinates
3,313741638944055297,Mon Mar 18 20:00:10 +0000 2013,"@OhSoFreshMusic haha, I just think he's a pret...","[[[-74.25909, 40.477399], [-74.25909, 40.91757...",place
4,313741646992924676,Mon Mar 18 20:00:12 +0000 2013,Drinks are so expensive on here,"[-80.1579172, 25.7763181]",coordinates
5,313741654869831680,Mon Mar 18 20:00:14 +0000 2013,"JAJAJAJAJAJAJAJA QUE MAL, QUE SIGO EN SHOCK TÚ...","[[[-124.482003, 32.528832], [-124.482003, 42.0...",place
6,313741662188888064,Mon Mar 18 20:00:15 +0000 2013,@LilRatchet__ @bigratchet__ oh .,"[-81.94830924, 34.45299573]",coordinates
7,313741668765556736,Mon Mar 18 20:00:17 +0000 2013,Lmfaoooo,"[-76.7056769, 39.1470045]",coordinates
8,313741675665178625,Mon Mar 18 20:00:19 +0000 2013,Ill leave it up to you baby fill in the blanks...,"[-81.33194438, 38.18019377]",coordinates
9,313741682199891971,Mon Mar 18 20:00:20 +0000 2013,I'm at @GrupoCitrus / Estudio Gráfico Multimed...,"[-107.37037182, 24.81529995]",coordinates


In [15]:
tw["coord"] = tw["coord"].apply(lambda x: re.findall(r"[-+]?\d*\.\d+|\d+",x) if isinstance(x,str) else None)
tw["coord"] = tw["coord"].apply(lambda x: [float(i) for i in x] if isinstance(x,list) else None)

In [16]:
"""assign a single coordinate to rows with location tag "place"""
indx = tw["loc_tag"]=="place"
tw_place = tw[indx]["coord"]
#tw_place = tw_place.apply(lambda x: x.replace("\"",""))
#tw_place =tw_place.apply(lambda x: ast.literal_eval(x))

#calculate the mean of the coordinates given; 
#because the coordinates are very close to each other, we can treat the surface as flat
def calc_mean(x):
    if x == None:
        return(None)
    else:
        lons = [x[i] for i in range(0,len(x),2)]
        lats = [x[i] for i in range(1,len(x),2)]
    lon = np.mean(lons)
    lat = np.mean(lats)
    return([lon,lat])

tw_place = tw_place.apply(calc_mean)
tw.loc[indx,"coord"] = tw_place

In [17]:
#convert time strings to datetime formate
tw["time"] = tw["time"].apply(lambda x: re.sub("\+0000\s","",str(x)) if re.search("\+0000\s",str(x)) else None)
tw["time"] = tw["time"].apply(lambda x: datetime.strptime(x,"%a %b %d %H:%M:%S %Y") if isinstance(x,str) else None)

In [19]:
#convert id-string to int
#tw["id"] = tw["id"].apply(lambda x: float(x) if re.search("^[0-9]{18}$",x) else None)

In [20]:
#remove first colum with full row
#del tw[0]

#remove all rows which contain None in any row
indx = tw.apply(lambda x: False if x.isnull().any() else True,axis=1)
tw = tw[indx]
print("Removed a total of "+ str(sum(~indx)) + " rows")

Removed a total of 49 rows


In [21]:
#Tokenize & Count tokens in tweet text
#from https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False,stopwords_rm=False,**kwargs):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    if stopwords_rm:
        tokens = [token for token in tokens if token not in kwargs["stop_words"]]
    return tokens

In [22]:
tw["tokens"] = tw["text"].apply(preprocess,lowercase=True)

/home/drosoneuro/anaconda3/envs/python_ml/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [23]:
#Count occurrence of each word
counts = Counter(itertools.chain.from_iterable(tw["tokens"]))

In [24]:
counts

Counter({'sticky': 2,
         'celebration': 5,
         'sopporto': 1,
         'unblock': 1,
         '#teamsingle': 1,
         '@starlightcasino': 1,
         'milwaukee': 9,
         '@alcivarmilton': 1,
         '@llct3rrall': 1,
         'http://t.co/peb4uatyzy': 1,
         'sleepppp': 1,
         'pole': 4,
         '@ryan033215': 1,
         'aswe': 1,
         '#thatbad': 1,
         'http://t.co/rg0meq9b': 1,
         'plátanos': 1,
         'ric': 1,
         'parcel': 1,
         'forums': 1,
         'otp': 1,
         'boreddd': 2,
         '@_immackkulate': 1,
         'amc': 7,
         '@nevakiss_n_tell': 1,
         '@krhumphreys12': 1,
         '@jeremykta': 1,
         'wr': 1,
         'woops': 1,
         'daibetic': 1,
         '1': 324,
         'jennifer': 2,
         'http://t.co/4xenckra': 1,
         'http://t.co/6hlmwffq': 1,
         'fortunately': 1,
         'drr': 1,
         '@sydthekidxoxo': 1,
         '@_cool_breezef1_': 1,
         'nerds': 2,
 

In [30]:
#remove punctuation and stopwords
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']
tw["tokens_stop"] = tw["text"].apply(preprocess,lowercase=True,
                              stopwords_rm = True, stop_words=stop)

/home/drosoneuro/anaconda3/envs/python_ml/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
tw

,id,time,text,coord,loc_tag,tokens,tokens_stop
0,313741620505890817,2013-03-18 20:00:06,got outstanding student of the semester :) sup...,"[-74.2041177, 40.9002544]",coordinates,"[got, outstanding, student, of, the, semester,...","[got, outstanding, student, semester, :), supe..."
1,313741627598450689,2013-03-18 20:00:07,Lol jk jk .,"[-74.98284917, 40.09872214]",coordinates,"[lol, jk, jk, .]","[lol, jk, jk]"
2,313741633277542400,2013-03-18 20:00:09,"Hey @julie_susanne, is it ok if I rent a few m...","[-74.18003082, 40.69879569]",coordinates,"[hey, @julie_susanne, ,, is, it, ok, if, i, re...","[hey, @julie_susanne, ok, rent, movies, i'm, g..."
3,313741638944055297,2013-03-18 20:00:10,"@OhSoFreshMusic haha, I just think he's a pret...","[-73.979681, 40.697488]",place,"[@ohsofreshmusic, haha, ,, i, just, think, he'...","[@ohsofreshmusic, haha, think, he's, pretty, a..."
4,313741646992924676,2013-03-18 20:00:12,Drinks are so expensive on here,"[-80.1579172, 25.7763181]",coordinates,"[drinks, are, so, expensive, on, here]","[drinks, expensive]"
5,313741654869831680,2013-03-18 20:00:14,"JAJAJAJAJAJAJAJA QUE MAL, QUE SIGO EN SHOCK TÚ...","[-119.306607, 37.2691745]",place,"[jajajajajajajaja, que, mal, ,, que, sigo, en,...","[jajajajajajajaja, que, mal, que, sigo, en, sh..."
6,313741662188888064,2013-03-18 20:00:15,@LilRatchet__ @bigratchet__ oh .,"[-81.94830924, 34.45299573]",coordinates,"[@lilratchet__, @bigratchet__, oh, .]","[@lilratchet__, @bigratchet__, oh]"
7,313741668765556736,2013-03-18 20:00:17,Lmfaoooo,"[-76.7056769, 39.1470045]",coordinates,[lmfaoooo],[lmfaoooo]
8,313741675665178625,2013-03-18 20:00:19,Ill leave it up to you baby fill in the blanks...,"[-81.33194438, 38.18019377]",coordinates,"[ill, leave, it, up, to, you, baby, fill, in, ...","[ill, leave, baby, fill, blanks, _____, #gregb..."
9,313741682199891971,2013-03-18 20:00:20,I'm at @GrupoCitrus / Estudio Gráfico Multimed...,"[-107.37037182, 24.81529995]",coordinates,"[i'm, at, @grupocitrus, /, estudio, gráfico, m...","[i'm, @grupocitrus, estudio, gráfico, multimed..."


In [32]:
#create bigrams
tw["bigrams"] = tw["tokens_stop"].apply(bigrams)

/home/drosoneuro/anaconda3/envs/python_ml/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [33]:
tw

,id,time,text,coord,loc_tag,tokens,tokens_stop,bigrams
0,313741620505890817,2013-03-18 20:00:06,got outstanding student of the semester :) sup...,"[-74.2041177, 40.9002544]",coordinates,"[got, outstanding, student, of, the, semester,...","[got, outstanding, student, semester, :), supe...",<generator object bigrams at 0x7fc314817410>
1,313741627598450689,2013-03-18 20:00:07,Lol jk jk .,"[-74.98284917, 40.09872214]",coordinates,"[lol, jk, jk, .]","[lol, jk, jk]",<generator object bigrams at 0x7fc3195373b8>
2,313741633277542400,2013-03-18 20:00:09,"Hey @julie_susanne, is it ok if I rent a few m...","[-74.18003082, 40.69879569]",coordinates,"[hey, @julie_susanne, ,, is, it, ok, if, i, re...","[hey, @julie_susanne, ok, rent, movies, i'm, g...",<generator object bigrams at 0x7fc31954c8e0>
3,313741638944055297,2013-03-18 20:00:10,"@OhSoFreshMusic haha, I just think he's a pret...","[-73.979681, 40.697488]",place,"[@ohsofreshmusic, haha, ,, i, just, think, he'...","[@ohsofreshmusic, haha, think, he's, pretty, a...",<generator object bigrams at 0x7fc315ac7360>
4,313741646992924676,2013-03-18 20:00:12,Drinks are so expensive on here,"[-80.1579172, 25.7763181]",coordinates,"[drinks, are, so, expensive, on, here]","[drinks, expensive]",<generator object bigrams at 0x7fc315ac7570>
5,313741654869831680,2013-03-18 20:00:14,"JAJAJAJAJAJAJAJA QUE MAL, QUE SIGO EN SHOCK TÚ...","[-119.306607, 37.2691745]",place,"[jajajajajajajaja, que, mal, ,, que, sigo, en,...","[jajajajajajajaja, que, mal, que, sigo, en, sh...",<generator object bigrams at 0x7fc31bdf1780>
6,313741662188888064,2013-03-18 20:00:15,@LilRatchet__ @bigratchet__ oh .,"[-81.94830924, 34.45299573]",coordinates,"[@lilratchet__, @bigratchet__, oh, .]","[@lilratchet__, @bigratchet__, oh]",<generator object bigrams at 0x7fc31bdf17d8>
7,313741668765556736,2013-03-18 20:00:17,Lmfaoooo,"[-76.7056769, 39.1470045]",coordinates,[lmfaoooo],[lmfaoooo],<generator object bigrams at 0x7fc31b9ce0a0>
8,313741675665178625,2013-03-18 20:00:19,Ill leave it up to you baby fill in the blanks...,"[-81.33194438, 38.18019377]",coordinates,"[ill, leave, it, up, to, you, baby, fill, in, ...","[ill, leave, baby, fill, blanks, _____, #gregb...",<generator object bigrams at 0x7fc31bdcee60>
9,313741682199891971,2013-03-18 20:00:20,I'm at @GrupoCitrus / Estudio Gráfico Multimed...,"[-107.37037182, 24.81529995]",coordinates,"[i'm, at, @grupocitrus, /, estudio, gráfico, m...","[i'm, @grupocitrus, estudio, gráfico, multimed...",<generator object bigrams at 0x7fc31b6f7f10>


In [34]:
#switch lat & lon position to work with reverse geocoder
tw["coord"] = tw["coord"].apply(lambda x: [x[1],x[0]] if len(x)>1 else None)

/home/drosoneuro/anaconda3/envs/python_ml/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [35]:
tw = tw[[i is not None for i in tw["coord"]]]

In [36]:
#get state location for each tweet
def state_lookup(x):
    coords =[(i,j) for i,j in x.values.flatten()]
    loc_info = rg.search(coords)
    states = [i["admin1"] for i in loc_info]
    return(states)
tw["state"] = state_lookup(tw["coord"])

Loading formatted geocoded file...


In [37]:
tw

,id,time,text,coord,loc_tag,tokens,tokens_stop,bigrams,state
0,313741620505890817,2013-03-18 20:00:06,got outstanding student of the semester :) sup...,"[40.9002544, -74.2041177]",coordinates,"[got, outstanding, student, of, the, semester,...","[got, outstanding, student, semester, :), supe...",<generator object bigrams at 0x7fc314817410>,New Jersey
1,313741627598450689,2013-03-18 20:00:07,Lol jk jk .,"[40.09872214, -74.98284917]",coordinates,"[lol, jk, jk, .]","[lol, jk, jk]",<generator object bigrams at 0x7fc3195373b8>,Pennsylvania
2,313741633277542400,2013-03-18 20:00:09,"Hey @julie_susanne, is it ok if I rent a few m...","[40.69879569, -74.18003082]",coordinates,"[hey, @julie_susanne, ,, is, it, ok, if, i, re...","[hey, @julie_susanne, ok, rent, movies, i'm, g...",<generator object bigrams at 0x7fc31954c8e0>,New Jersey
3,313741638944055297,2013-03-18 20:00:10,"@OhSoFreshMusic haha, I just think he's a pret...","[40.697488, -73.979681]",place,"[@ohsofreshmusic, haha, ,, i, just, think, he'...","[@ohsofreshmusic, haha, think, he's, pretty, a...",<generator object bigrams at 0x7fc315ac7360>,New York
4,313741646992924676,2013-03-18 20:00:12,Drinks are so expensive on here,"[25.7763181, -80.1579172]",coordinates,"[drinks, are, so, expensive, on, here]","[drinks, expensive]",<generator object bigrams at 0x7fc315ac7570>,Florida
5,313741654869831680,2013-03-18 20:00:14,"JAJAJAJAJAJAJAJA QUE MAL, QUE SIGO EN SHOCK TÚ...","[37.2691745, -119.306607]",place,"[jajajajajajajaja, que, mal, ,, que, sigo, en,...","[jajajajajajajaja, que, mal, que, sigo, en, sh...",<generator object bigrams at 0x7fc31bdf1780>,California
6,313741662188888064,2013-03-18 20:00:15,@LilRatchet__ @bigratchet__ oh .,"[34.45299573, -81.94830924]",coordinates,"[@lilratchet__, @bigratchet__, oh, .]","[@lilratchet__, @bigratchet__, oh]",<generator object bigrams at 0x7fc31bdf17d8>,South Carolina
7,313741668765556736,2013-03-18 20:00:17,Lmfaoooo,"[39.1470045, -76.7056769]",coordinates,[lmfaoooo],[lmfaoooo],<generator object bigrams at 0x7fc31b9ce0a0>,Maryland
8,313741675665178625,2013-03-18 20:00:19,Ill leave it up to you baby fill in the blanks...,"[38.18019377, -81.33194438]",coordinates,"[ill, leave, it, up, to, you, baby, fill, in, ...","[ill, leave, baby, fill, blanks, _____, #gregb...",<generator object bigrams at 0x7fc31bdcee60>,West Virginia
9,313741682199891971,2013-03-18 20:00:20,I'm at @GrupoCitrus / Estudio Gráfico Multimed...,"[24.81529995, -107.37037182]",coordinates,"[i'm, at, @grupocitrus, /, estudio, gráfico, m...","[i'm, @grupocitrus, estudio, gráfico, multimed...",<generator object bigrams at 0x7fc31b6f7f10>,Sinaloa


# 2. Build model

## 2.1 Multiple linear regression 

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()


In [40]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

In [42]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [63]:
len(twenty_train.data)
tw <- tw[::50]

TypeError: bad operand type for unary -: 'Timestamp'

In [47]:
X_train_counts = count_vect.fit_transform(tw["text"])

In [62]:
X_train_counts.shape

<37621x57514 sparse matrix of type '<class 'numpy.int64'>'
	with 338646 stored elements in Compressed Sparse Row format>

In [51]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(37621, 57514)

In [52]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(37621, 57514)

In [55]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, tw["text"])

MemoryError: 